## V1 program

```javascript
var a
func f() {
    a = 5
    var b
    b = 6
}
func g() {
    var c
    c = 7
}
f()
g()
```

## V1 Grammar

```yaml
program: (statement | func_def)*
statement: definition | attribution | func_call
definition: "var" NAME
attribution: NAME "=" NUMBER
func_call: NAME()
func_def: "func" NAME() "{" statement* "}"
```

## Analyzer

In [11]:
program = """
var a
func f() {
    a = 5
    var b
    b = 6
}
func g() {
    var c
    c = 7
}
f()
g()
"""

In [12]:
import rich

def analyze(p):
    memory_adress = 0
    for line_number, line in enumerate(p.strip().split('\n')):
        match line.split():
            case ['var', name]:
                # adicionar variável na tabela de símbolos global ou local
                # dependendo se estiver fora 
                if name in symbol_table:
                    # variáveis locais não podem ter o mesmo nome de globais
                    rich.print('[red1]redefined:', name)
                else:
                    # o endereço de variáveis locais não interessa (colocar 42)
                    symbol_table[name] = memory_adress
                    memory_adress += 1
            case [name, '=', number]:
                # a atribuição é válida para variáveis globais ou para locais
                if name not in symbol_table:
                    rich.print('[red1]variable unknown:', name)
            case ['func', name, '{']:
                if name in function_table:
                    rich.print('[red1]function redefined:', name)
                else:
                    function_table[name] = line_number
            case ['}']:
                pass
            case [name] if name.endswith('()'):
                if name not in function_table:
                    rich.print('[red1]function unknown:', name)
            case _:
                rich.print('[red1]unmatched:', line)

symbol_table = {}
function_table = {}
analyze(program)
print('symbol_table:', symbol_table)
print('function_table:', function_table)

symbol_table: {'a': 0, 'b': 1, 'c': 2}
function_table: {'f()': 1, 'g()': 6}


In [13]:
def execute(p):
    pc = 0 # program counter
    lines = p.strip().split('\n')
    while pc < len(lines):
        match lines[pc].split():
            case ['var', name]:
                # se for definição local
                #     incrementa pilha (memory.append(0))
                #     adiciona name no quadro de ativação atual com endereço do topo da pilha
                pass
            case [name, '=', number]:
                # atribuição em variável global ou local?
                address = symbol_table[name]
                static_memory[address] = int(number)
                print(name, 'at address', address, 'receives', number)
            case ['func', name, '{']:
                while lines[pc] != '}':
                    pc += 1
            case [name] if name.endswith('()'):
                call_stack.append(pc)
                print(name, 'called in line', pc)
                pc = function_table[name]
            case ['}']:
                # para o último quadro de ativação:
                #     limpar a parte de variáveis locais da memória
                #     eliminar último quadro de ativação
                pc = call_stack.pop()
                print('return to line', pc)
        pc += 1
    print('program ended\n')

# reutilizar a tabela de símbolos local para controle das variáveis automáticas
call_stack = []
static_memory = [0] * len(symbol_table) # tamanho pré-alocado é para as variáveis globais apenas
execute(program)
print('static_memory', static_memory)
print('call_stack', call_stack)

f() called in line 10
a at address 0 receives 5
b at address 1 receives 6
return to line 10
g() called in line 11
c at address 2 receives 7
return to line 11
program ended

static_memory [5, 6, 7]
call_stack []
